In [1]:
# Import required libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import defaultdict
import seaborn as sns

sns.set()

In [2]:
# Read the dataset
data = pd.read_csv('Online Retail Data Set.csv', encoding='latin1')

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### Observations

'Description' and 'CustomerID' columns have null values.

Lets check if we can fill CustomerID from InvoiceNo.

In [5]:
# Check one-to-many mapping of some columns to check for data correctness

def isOneToMany(df, col1, col2):
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    print(first == 1)

In [6]:
# Check if each InvoiceNo is unique to a CustomerID
isOneToMany(data, 'InvoiceNo', 'CustomerID')

True


Considering we are not able to fill up CustomerID, and we have enough data, we will drop observations with NaN CustomerID

In [7]:
# Delete rows with missing CustomerID
data = data.drop(data[data.CustomerID.isnull()].index).reset_index(drop=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406829 entries, 0 to 406828
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 24.8+ MB


Delete missing CustomerId has removed all missing Description rows too.

In [9]:
# Convert "InvoiceNo" to a string type series
data['InvoiceNo'] = data.InvoiceNo.astype('str')

In [10]:
# Convert "Description" to a string type series and remove extra whitespaces
data['Description'] = data.Description.astype('str')
data['Description'] = data.Description.str.strip()

In [11]:
# Check for data correctness based on one to many relationship between some columns

isOneToMany(data,'InvoiceNo','Country')
isOneToMany(data,'StockCode','Description')

True
False


StockCode and Description should have one-to-many relationship. Which means each StockCode needs to uniquely represent an item Description, but each Description may not have just one StockCode. But our dataset has multiple Description for same StockCode.

Let us analyze this:

In [12]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [13]:
# Check duplicacy of description in StockCode

products = data.groupby(['StockCode','Description'],as_index=False).count()
products = products.loc[:,['StockCode','Description']]
stockCount = products.StockCode.value_counts().to_frame()
stockList = list(stockCount[stockCount.StockCode>1].index) #List of stockCode with more than 1 Description
products[products.StockCode.isin(stockList)].head(10)

    StockCode                          Description
42     16156L                        WRAP CAROUSEL
43     16156L                       WRAP, CAROUSEL
101    17107D         FLOWER FAIRY 5 DRAWER LINERS
102    17107D    FLOWER FAIRY 5 SUMMER DRAW LINERS
103    17107D  FLOWER FAIRY,5 SUMMER B'DRAW LINERS
121     20622                   VIP PASSPORT COVER
122     20622                    VIPPASSPORT COVER
174     20725              LUNCH BAG RED RETROSPOT
175     20725                 LUNCH BAG RED SPOTTY
267     20914  SET/5 RED RETROSPOT LID GLASS BOWLS


Each StockCode does identify unique item, but there are some data entry errors in the description.

In [14]:
# Remove duplicate entries
products.drop_duplicates('StockCode', keep='first', inplace=True)

In [15]:
# Correct the mapping of StockCode and Description

df = pd.merge(data,products,on='StockCode', how='left',validate='many_to_one')
df.drop('Description_x',axis=1,inplace=True)
df.rename(columns={'Description_y':'Description'},inplace=True)
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,01-12-2010 08:26,2.55,17850.0,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,8,01-12-2010 08:26,2.75,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


In [16]:
# Check for StockCode and Description mapping
isOneToMany(df,'StockCode','Description')

True


The data description in the data source mentions that the StockCode is a 5 digit integer. But we observed StockCodes with length different from 5 digits. We will analyse them to check for any discrepancies.

In [17]:
df[(df.StockCode.str.len())==6].sort_values(by='StockCode').head(10)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
96416,548491,10123C,1,31-03-2011 13:14,0.65,14064.0,United Kingdom,HEARTS WRAPPING TAPE
4165,536863,10123C,1,03-12-2010 11:19,0.65,17967.0,United Kingdom,HEARTS WRAPPING TAPE
72675,545655,10123C,3,04-03-2011 13:58,0.65,14670.0,United Kingdom,HEARTS WRAPPING TAPE
8987,537382,10124A,4,06-12-2010 13:13,0.42,16710.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
53974,543397,10124A,3,08-02-2011 10:32,0.42,17859.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
92138,547881,10124A,1,27-03-2011 16:08,0.42,13110.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
83288,546913,10124A,4,17-03-2011 20:18,0.42,15861.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
333228,574686,10124A,4,06-11-2011 13:00,0.42,17608.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
83287,546913,10124G,4,17-03-2011 20:18,0.42,15861.0,United Kingdom,ARMY CAMO BOOKCOVER TAPE
270614,568938,10124G,4,29-09-2011 14:46,0.42,17220.0,United Kingdom,ARMY CAMO BOOKCOVER TAPE


In [18]:
df[(df.StockCode.str.len())<5].sort_values(by='StockCode').head(10)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
201605,561631,C2,1,28-07-2011 14:27,50.0,14911.0,EIRE,CARRIAGE
25993,539688,C2,1,21-12-2010 11:00,150.0,12678.0,France,CARRIAGE
155387,555858,C2,1,07-06-2011 14:12,50.0,14911.0,EIRE,CARRIAGE
66561,544934,C2,1,25-02-2011 09:01,50.0,13345.0,United Kingdom,CARRIAGE
389253,579910,C2,1,01-12-2011 08:52,50.0,14911.0,EIRE,CARRIAGE
26731,539984,C2,1,23-12-2010 14:58,50.0,14911.0,EIRE,CARRIAGE
179328,558908,C2,1,05-07-2011 09:06,50.0,13338.0,United Kingdom,CARRIAGE
242705,566261,C2,1,11-09-2011 12:46,50.0,13372.0,United Kingdom,CARRIAGE
315365,572969,C2,1,27-10-2011 10:46,50.0,13363.0,United Kingdom,CARRIAGE
27953,C540164,C2,-1,05-01-2011 12:02,50.0,14911.0,EIRE,CARRIAGE


There are some 6-digit codes in StockCodes which consist of 5 digits and last one an alphabet. The last albhabet seems to distinguish the variants of the item specified by 5-digit code. So they are valid.

The StockCodes with lesser than 5 digits seem to be shipping charges/discounts/other incidental charges related to an order. As these are not related to companies sales, we will delete them.

In [19]:
# Drop observations with incidental charges

df = df.drop(df[(df.StockCode.str.len())<5].index).reset_index(drop=True)

In [20]:
df.describe(include='all')

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
count,404921,404921,404921.000000,404921,404921.000000,404921.000000,404921,404921
unique,21799,3677,NaN,20145,NaN,NaN,37,3652
top,576339,85123A,NaN,14-11-2011 15:27,NaN,NaN,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
freq,541,2077,NaN,541,NaN,NaN,361267,2077
mean,NaN,NaN,12.105470,NaN,2.901451,15295.158048,NaN,NaN
std,NaN,NaN,249.233858,NaN,4.431232,1710.414161,NaN,NaN
min,NaN,NaN,-80995.000000,NaN,0.000000,12346.000000,NaN,NaN
25%,NaN,NaN,2.000000,NaN,1.250000,13969.000000,NaN,NaN
50%,NaN,NaN,5.000000,NaN,1.950000,15159.000000,NaN,NaN
75%,NaN,NaN,12.000000,NaN,3.750000,16794.000000,NaN,NaN


#### Observations

1. Quantity values look incorrect
2. In some cases, the UnitPrice is 0. Some more investigation is needed to check data correctness
3. 89% of the sales is coming from UK
4. Number of unique items for StockCode and Description is different indicating one-to-many relationship.


The CustomerID appears as a float in the dataset. As it is an identifier, we will append the letter C to it to make sure it is not treated like a float/int.

In [21]:
df['CustomerID'] = 'C' + df.CustomerID.astype('int').astype('str')
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,01-12-2010 08:26,2.55,C17850,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,01-12-2010 08:26,3.39,C17850,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,8,01-12-2010 08:26,2.75,C17850,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,01-12-2010 08:26,3.39,C17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,01-12-2010 08:26,3.39,C17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


#### Check large Quantity values for data correctness

In [22]:
# Check negative quanitities

df[(df.Quantity<1) & (~df.InvoiceNo.str.startswith('C'))]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description


All the negative quantities are cancelled orders.

In [23]:
# Check for unit quantity of more than 4000
df[(abs(df.Quantity)>4000)]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
3095,C536757,84347,-9360,02-12-2010 14:23,0.03,C15838,United Kingdom,ROTATING SILVER ANGELS T-LIGHT HLDR
37950,541431,23166,74215,18-01-2011 10:01,1.04,C12346,United Kingdom,MEDIUM CERAMIC TOP STORAGE JAR
37955,C541433,23166,-74215,18-01-2011 10:17,1.04,C12346,United Kingdom,MEDIUM CERAMIC TOP STORAGE JAR
146078,554868,22197,4300,27-05-2011 10:52,0.72,C13135,United Kingdom,POPCORN HOLDER
314018,573008,84077,4800,27-10-2011 12:26,0.21,C12901,United Kingdom,WORLD WAR 2 GLIDERS ASSTD DESIGNS
377094,578841,84826,12540,25-11-2011 15:57,0.00,C13256,United Kingdom,ASSTD DESIGN 3D PAPER STICKERS
404447,581483,23843,80995,09-12-2011 09:15,2.08,C16446,United Kingdom,"PAPER CRAFT , LITTLE BIRDIE"
404448,C581484,23843,-80995,09-12-2011 09:27,2.08,C16446,United Kingdom,"PAPER CRAFT , LITTLE BIRDIE"


The two largest quantities of 80995 and 74215 units are part of cancelled orders. The quantity of 12540 has 0 UnitPrice and is the only item in the order. So we will delete it.

In [24]:
df = df.drop(df[df.Quantity==12540].index).reset_index(drop=True)
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,01-12-2010 08:26,2.55,C17850,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,01-12-2010 08:26,3.39,C17850,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,8,01-12-2010 08:26,2.75,C17850,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,01-12-2010 08:26,3.39,C17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,01-12-2010 08:26,3.39,C17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


In [25]:
# Check one-to-many mapping between CustomerID and Country

isOneToMany(df, 'CustomerID', 'Country')

False


In the given dataset, some of the CustomerID are linked with 2 countries. As per the data attribute description: 'Country' column is the name of the country where each customer resides. But we don't have any information on how is this data being captured. Is it through IP address of the country while creating account, or may be based on the shipping address, or may be something else.

Logically, each CustomerID should be linked to one country only. The reason for having more than one country could be:
1. Data entry error
2. Customer has moved to another country, and has got the address changed in his account
3. In case this attribute reflects the shipping address, the customer has shipped the order to an address different from his own. 
4. In case this attribute is captured through the IP address while ordering, the customer might be ordering while travelling to another country.

Further analysis of data does not make it clear what is the reason behind 2 countries for a CustomerID, so for now, we are not making any changes in the CustomerID and country linkage.

## Feature engineering

At this stage, we can derive following features from the already existing ones to aid in our analysis.
1. CancelledOrder containing boolean values, 1 if order was cancelled, 0 otherwise
2. InternationalOrders containing boolean values, 0 if order came from UK, 1 if the order came from outside UK
3. TotalPrice containing float values = unitPrice * Quantity

??Avg per unit price?/

grouping of item description



In [26]:
# Add new column with bool value: 0 for not cancelled order, 1 for cancelled order
df['CancelledOrder'] = df.InvoiceNo.str.startswith('C').astype('int')

In [27]:
df.Country.value_counts()

Country
United Kingdom          361266
Germany                   9096
France                    8168
EIRE                      7380
Spain                     2468
Netherlands               2330
Belgium                   1971
Switzerland               1844
Portugal                  1436
Australia                 1257
Norway                    1060
Italy                      783
Channel Islands            753
Finland                    653
Cyprus                     619
Sweden                     437
Austria                    387
Denmark                    375
Japan                      355
Poland                     336
USA                        291
Israel                     250
Unspecified                244
Singapore                  215
Iceland                    182
Canada                     150
Greece                     142
Malta                      123
United Arab Emirates        67
European Community          58
RSA                         57
Lebanon                     45


In [28]:
# Delete the countries 'Unspecified' and 'European Community'
df = df.drop(df[df.Country.isin(['Unspecified','European Community'])].index).reset_index(drop=True)

In [29]:
# Add new column 'InternationalOrders' with bool values, 0 for United Kingdom, and 1 for other countries
df['InternationalOrders'] = (df.Country!='United Kingdom').astype('int')

In [30]:
# Add a column for total value for each product in each order
df['TotalPrice'] = df.Quantity*df.UnitPrice

In [31]:
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description,CancelledOrder,InternationalOrders,TotalPrice
0,536365,85123A,6,01-12-2010 08:26,2.55,C17850,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER,0,0,15.30
1,536365,71053,6,01-12-2010 08:26,3.39,C17850,United Kingdom,WHITE METAL LANTERN,0,0,20.34
2,536365,84406B,8,01-12-2010 08:26,2.75,C17850,United Kingdom,CREAM CUPID HEARTS COAT HANGER,0,0,22.00
3,536365,84029G,6,01-12-2010 08:26,3.39,C17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE,0,0,20.34
4,536365,84029E,6,01-12-2010 08:26,3.39,C17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,0,0,20.34


In [32]:
df.to_csv('df.csv')